# Data Preprocessing Requirements for the model code

##### Inputs
In order to test Faster R-CNN, we need below Inputs:
Image File Name and Location
Bounding Box details - Xmin, Ymin, Xmax, Ymax
Class name - person, cat, dog etc

##### Data source
As part of this project, we used COCO dataset and Y2BB dataset.
These inputs are filtered from Y2BB and COCO Datasets and saved in a meta data or annotation file, which is a CSV or XML or JSON.

##### Data Format constraints for the model
We faced an issue while using those datasets in the model training/testing.
This was due to Faster R-CNN model code needing data in a certain format.
e.g., 
![title](Faster-R-CNN-DataSet.png)


##### Solution
We decided to preprocess the source meta data file.
e.g. COCO dataset and Y2BB dataset
For the uniformity, we chose 2 classes - Airplane and Train

##### Important point for Y2BB dataset
Faster R-CNN model input needs exact co-ordinates for the bounding box.
Hence we needed to scale coordinates of Y2BB metadata feed file by using image height and width respectively for Y and X coordinates.

In [1]:
import pandas as pd
import os
import cv2

In [2]:
def pre_process_coco_data(csvSourceFile, evalType, className, imageFolder):
    input_data = pd.read_csv(csvSourceFile)
    class_present = input_data.loc[input_data['present'] == 1]
    print(f'class_present = {class_present.shape[0]}')
    class_absent = input_data.loc[input_data['present'] == 0]
    print(f'class_absent = {class_absent.shape[0]}')
    
    data = pd.DataFrame()
    data['format'] = input_data['file_name']

    # as the images are in outputFolder folder, add location of the outputFolder before the image name
    for i in range(data.shape[0]):
        data['format'][i] = '' + str(imageFolder) + str(data['format'][i]) + ''
        #data['format'][i] = '' + str(data['format'][i]) + ''

    # add xmin, ymin, xmax, ymax and class as per the format required
    for i in range(data.shape[0]):
        class_name = input_data['category'][i]
        if input_data['x_min'][i] == -1:
            class_name = 'bg'
        data['format'][i] = data['format'][i] + ',' + str(input_data['x_min'][i]) + ',' + str(input_data['y_min'][i]) + ',' + str(input_data['x_max'][i]) + ',' + str(input_data['y_max'][i]) + ',' + class_name

    outputFileName = 'coco_' + evalType + "_" + className + '_annotate.txt'
    data.to_csv(outputFileName, header=None, index=None, sep=' ')
    return outputFileName

In [3]:
def pre_process_y2bb_data(csvSourceFile, evalType, className, imageFolder):
    input_data = pd.read_csv(csvSourceFile)
    class_present = input_data.loc[input_data['ObjectPresent'] == 1]
    print(f'class_present = {class_present.shape[0]}')
    class_absent = input_data.loc[input_data['ObjectPresent'] == 0]
    print(f'class_absent = {class_absent.shape[0]}')
    
    data = pd.DataFrame()
    data['format'] = input_data['ImageID'] 
    # as the images are in outputFolder folder, add location of the outputFolder before the image name
    for i in range(data.shape[0]):
        data['format'][i] = '' + str(imageFolder) + str(data['format'][i]) + ''
        #data['format'][i] = '' + str(data['format'][i]) + ''

    # add xmin, ymin, xmax, ymax and class as per the format required
    for i in range(data.shape[0]):
        ImageID = data['format'][i]
        print(f'ImageID = {str(ImageID)}')
        img = cv2.imread(str(ImageID))
        (height,width,_) = img.shape
        print(f'height={height}, width = {width}')

        class_name = input_data['ClassName'][i]
        if input_data['Xmin'][i] == -1:
            class_name = 'bg'
            Xmin = -1
            Xmax = -1
            Ymin = -1
            Ymax = -1
        else:
            Xmin = input_data['Xmin'][i] * width
            Xmax = input_data['Xmax'][i] * width
            Ymin = input_data['Ymin'][i] * height
            Ymax = input_data['Ymax'][i] * height
        data['format'][i] = data['format'][i] + ',' + str(Xmin) + ',' + str(Ymin) + ',' + str(Xmax) + ',' + str(Ymax) + ',' + class_name

    outputFileName = 'y2bb_' + evalType + "_" + className + '_annotate.txt'
    data.to_csv(outputFileName, header=None, index=None, sep=' ')
    return outputFileName

#### Preprocess Train and Test metadata CSV Files for COCO 'Train' class

In [5]:
csvSourceFile = 'COCO_extracted_dataset/train_for_train/train_for_trainset.csv'
evalType = 'training'
className = 'Train'
imageFolder = 'COCO_extracted_dataset/train_for_train/'
outputFileName = pre_process_coco_data(csvSourceFile,evalType, className,imageFolder)
print(f'outputFileName = {outputFileName}')

class_present = 950
class_absent = 50
outputFileName = coco_training_Train_annotate.txt


In [6]:
train_vehicle = pd.read_csv(outputFileName)
train_vehicle.head()

,COCO_extracted_dataset/train_for_train/COCO_train2014_000000032773.jpg,271.63,192.68,435.04,373.76,train
0,COCO_extracted_dataset/train_for_train/COCO_tr...,0.00,240.28,355.24,605.58,train
1,COCO_extracted_dataset/train_for_train/COCO_tr...,94.51,197.80,467.99,237.53,train
2,COCO_extracted_dataset/train_for_train/COCO_tr...,0.95,133.22,319.91,336.62,train
3,COCO_extracted_dataset/train_for_train/COCO_tr...,115.42,0.00,638.56,362.43,train
4,COCO_extracted_dataset/train_for_train/COCO_tr...,67.16,6.29,574.53,548.50,train


In [12]:
csvSourceFile = 'COCO_extracted_dataset/train_for_validation/train_for_validation_set.csv'
evalType = 'validation'
className = 'Train'
imageFolder = 'COCO_extracted_dataset/train_for_validation/'
outputFileName = pre_process_coco_data(csvSourceFile,evalType, className,imageFolder)
print(f'outputFileName = {outputFileName}')

class_present = 200
class_absent = 0
outputFileName = coco_validation_Train_annotate.txt


In [13]:
train_vehicle = pd.read_csv(outputFileName)
train_vehicle.head()

,COCO_extracted_dataset/train_for_validation/COCO_val2014_000000221190.jpg,285.47,238.49,441.75,270.02,train
0,COCO_extracted_dataset/train_for_validation/CO...,4.31,110.02,628.85,390.47,train
1,COCO_extracted_dataset/train_for_validation/CO...,97.87,144.89,498.01,356.95,train
2,COCO_extracted_dataset/train_for_validation/CO...,371.80,397.83,479.60,486.62,train
3,COCO_extracted_dataset/train_for_validation/CO...,66.68,169.52,522.09,328.33,train
4,COCO_extracted_dataset/train_for_validation/CO...,0.00,0.00,500.00,332.00,train


In [8]:
csvSourceFile = 'COCO_extracted_dataset/airplane_for_validation/airplane_for_validation_set.csv'
evalType = 'validation'
className = 'Airplane'
imageFolder = 'COCO_extracted_dataset/airplane_for_validation/'
outputFileName = pre_process_coco_data(csvSourceFile,evalType, className,imageFolder)
print(f'outputFileName = {outputFileName}')

class_present = 200
class_absent = 0
outputFileName = coco_validation_Airplane_annotate.txt


In [9]:
train_vehicle = pd.read_csv(outputFileName)
train_vehicle.head()

,COCO_extracted_dataset/airplane_for_validation/COCO_val2014_000000081922.jpg,218.07,183.32,377.0,230.22,airplane
0,COCO_extracted_dataset/airplane_for_validation...,185.77,189.62,197.57,198.28,airplane
1,COCO_extracted_dataset/airplane_for_validation...,232.67,148.37,425.99,284.59,airplane
2,COCO_extracted_dataset/airplane_for_validation...,241.46,229.56,454.16,299.52,airplane
3,COCO_extracted_dataset/airplane_for_validation...,0.00,161.86,563.87,401.16,airplane
4,COCO_extracted_dataset/airplane_for_validation...,35.87,39.61,182.24,130.01,airplane


In [6]:
csvSourceFile = 'COCO_extracted_dataset/train_for_test/train_for_test_set.csv'
evalType = 'testing'
className = 'Train'
imageFolder = 'COCO_extracted_dataset/train_for_test/'
outputFileName = pre_process_coco_data(csvSourceFile,evalType,className,imageFolder)
print(f'outputFileName = {outputFileName}')

class_present = 300
class_absent = 0
outputFileName = coco_testing_Train_annotate.txt


In [7]:
train_vehicle_test = pd.read_csv(outputFileName)
train_vehicle_test.head()

,COCO_extracted_dataset/train_for_test/COCO_val2014_000000542257.jpg,293.39,173.66,488.63,335.46000000000004,train
0,COCO_extracted_dataset/train_for_test/COCO_val...,2.42,70.19,590.53,243.64,train
1,COCO_extracted_dataset/train_for_test/COCO_val...,207.41,452.76,320.65,592.37,train
2,COCO_extracted_dataset/train_for_test/COCO_val...,4.30,22.60,637.13,476.56,train
3,COCO_extracted_dataset/train_for_test/COCO_val...,19.15,74.67,575.34,362.82,train
4,COCO_extracted_dataset/train_for_test/COCO_val...,1.13,174.76,526.56,460.74,train


#### Preprocess Train and Test metadata CSV Files for COCO 'AirPlane' class

In [8]:
csvSourceFile = 'COCO_extracted_dataset/airplane_for_train/airplane_for_trainset.csv'
evalType = 'training'
className = 'Airplane'
imageFolder = 'COCO_extracted_dataset/airplane_for_train/'
outputFileName = pre_process_coco_data(csvSourceFile,evalType, className,imageFolder)
print(f'outputFileName = {outputFileName}')

class_present = 950
class_absent = 50
outputFileName = coco_training_Airplane_annotate.txt


In [9]:
airplane_vehicle = pd.read_csv(outputFileName)
airplane_vehicle.head()

,COCO_extracted_dataset/airplane_for_train/COCO_train2014_000000393221.jpg,205.59,52.94,208.51,57.93,airplane
0,COCO_extracted_dataset/airplane_for_train/COCO...,311.71,156.23,487.78,246.78,airplane
1,COCO_extracted_dataset/airplane_for_train/COCO...,97.52,124.33,556.93,297.70,airplane
2,COCO_extracted_dataset/airplane_for_train/COCO...,178.79,126.74,632.05,345.16,airplane
3,COCO_extracted_dataset/airplane_for_train/COCO...,337.49,138.70,530.63,221.92,airplane
4,COCO_extracted_dataset/airplane_for_train/COCO...,57.21,85.76,537.19,539.43,airplane


In [10]:
csvSourceFile = 'COCO_extracted_dataset/airplane_for_test/airplane_for_test_set.csv'
evalType = 'testing'
className = 'Airplane'
imageFolder = 'COCO_extracted_dataset/airplane_for_test/'
outputFileName = pre_process_coco_data(csvSourceFile,evalType,className,imageFolder)
print(f'outputFileName = {outputFileName}')

class_present = 300
class_absent = 0
outputFileName = coco_testing_Airplane_annotate.txt


In [11]:
airplane_vehicle_test = pd.read_csv(outputFileName)
airplane_vehicle_test.head()

,COCO_extracted_dataset/airplane_for_test/COCO_val2014_000000137950.jpg,4.64,32.18,535.6899999999999,173.3,airplane
0,COCO_extracted_dataset/airplane_for_test/COCO_...,18.34,70.11,640.00,339.78,airplane
1,COCO_extracted_dataset/airplane_for_test/COCO_...,490.65,224.54,518.18,244.25,airplane
2,COCO_extracted_dataset/airplane_for_test/COCO_...,204.07,201.76,259.93,230.58,airplane
3,COCO_extracted_dataset/airplane_for_test/COCO_...,92.58,340.76,170.22,407.41,airplane
4,COCO_extracted_dataset/airplane_for_test/COCO_...,197.39,99.51,640.00,305.53,airplane


#### Preprocess Train and Test metadata CSV Files for Y2BB 'Train' class

In [15]:
csvSourceFile = 'Y2BB_extracted_dataset/train_for_train/YTBB_train_traindata_metadata_v1.csv'
evalType = 'training'
className = 'Train'
imageFolder = 'Y2BB_extracted_dataset/train_for_train/'
outputFileName = pre_process_y2bb_data(csvSourceFile,evalType, className,imageFolder)
print(f'outputFileName = {outputFileName}')

class_present = 950
class_absent = 50
ImageID = Y2BB_extracted_dataset/train_for_train/15+-6-4soyVvC4+01+0043000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+10X4sACw0Bk+01+0081000+1.jpg
height=288, width = 384
ImageID = Y2BB_extracted_dataset/train_for_train/15+1gXCV_X8Hw8+00+0008000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+3I5-oJDuyj8+00+0009000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/train_for_train/15+0qh9LTeOGY8+00+0046000+1.jpg
height=144, width = 192
ImageID = Y2BB_extracted_dataset/train_for_train/15+2iqay12cCJc+01+0110000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_train/15+3T1_hat2TUg+04+0265000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+0vfg9ZKYA2s+00+0005000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+3j3YyvV_OZU+00+0026000+1.jpg
height=360, width = 480
ImageID = Y2BB_e

height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_train/15+1FtR9FcUUns+00+0061000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+0vfg9ZKYA2s+00+0009000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+0BSfrXvRmLk+00+0071000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/train_for_train/15+0hxGNJtSd80+00+0020000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_train/15+2tKPCpzMY80+00+0001000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+2qs-ClwgZjY+01+0037000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_train/15+0ivt9h4aXps+00+0028000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+1OTWig_5hzg+00+0024000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+2X4c-0Ctmmw+02+0044000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_datas

height=360, width = 450
ImageID = Y2BB_extracted_dataset/train_for_train/15+3HiVmE5mjNo+01+0084000+1.jpg
height=288, width = 384
ImageID = Y2BB_extracted_dataset/train_for_train/15+3X68YQk3WpE+00+0010000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+3oFYQY3IG-0+00+0010000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+-ZWE-igmdLU+01+0046000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+28DJ1IafNdA+04+0411000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+3HiVmE5mjNo+01+0091000+1.jpg
height=288, width = 384
ImageID = Y2BB_extracted_dataset/train_for_train/15+0iIb6Q7aKvc+00+0006000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/train_for_train/15+-025GNtJ0oI+01+0104000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+0IGlkRhZ3N0+00+0010000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_datas

height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_train/15+-GHxhfOfLKI+00+0026000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_train/15+07yqmNxwcYQ+00+0029000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+-BwXEDixxNM+00+0015000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+05X3-EAJ99s+01+0038000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_train/15+1USRwMcH0Fc+01+0094000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+235NI9CNNYs+01+0064000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_train/15+1tvjAf_uoiQ+00+0014000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+2qy1Sjwf54I+01+0040000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+0Z2XnkJAIQ8+00+0052000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_datase

height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+-yMI3k70zIU+02+0101000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+3qp1JxIVwOI+00+0002000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_train/15+-wk7J5aKLB4+01+0039000+1.jpg
height=272, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+00iw45dQOBA+00+0006000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_train/15+-m6OhoJ4Q48+01+0036000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+2KGfQgVH85c+00+0046000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_train/15+2EU0pbb9rNE+02+0074000+1.jpg
height=360, width = 640
ImageID = Y2BB_extracted_dataset/train_for_train/15+0bo7VByvo0Y+00+0108000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+-771AUMRGi0+01+0036000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_datase

height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+2zRN1VZhBqU+00+0118000+1.jpg
height=360, width = 640
ImageID = Y2BB_extracted_dataset/train_for_train/15+2MnbKb4krcY+00+0020000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+070CuYCc2Xw+02+0083000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_train/15+2FMue4ul6oc+01+0136000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+0G9gcbaLaHo+00+0015000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+3FmiqUY3PSU+00+0009000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+3LTSe1BRCqo+01+0082000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+0qih7ASxWdM+00+0003000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+0eWtYZVw73g+00+0014000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_datase

height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+2ZERAlzfLds+00+0125000+1.jpg
height=360, width = 636
ImageID = Y2BB_extracted_dataset/train_for_train/15+2HaRwvJEOYA+02+0099000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+2wT-x0YyivY+01+0309000+1.jpg
height=360, width = 640
ImageID = Y2BB_extracted_dataset/train_for_train/15+3LId9qziO1k+00+0024000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_train/15+3P_b80TKV_0+00+0007000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+3P_b80TKV_0+00+0013000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+1x_1Gg-VW_c+00+0013000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+2lYnwkOVMlg+00+0076000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+2vp0W1OKBPE+00+0049000+1.jpg
height=240, width = 350
ImageID = Y2BB_extracted_datase

height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_train/15+3X3-ZoE3yHI+01+0037000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_train/15+3Sx2QvjLSbk+02+0225000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+22NHRnlFer4+00+0160000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+3ID3J5sOqaQ+00+0015000+1.jpg
height=360, width = 636
ImageID = Y2BB_extracted_dataset/train_for_train/15+2-Xaynl3qbQ+00+0009000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_train/15+3lF47TszHME+03+0243000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_train/15+1KaJH3b51VM+00+0014000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_train/15+1MesnU0kno8+00+0044000+1.jpg
height=318, width = 426
ImageID = Y2BB_extracted_dataset/train_for_train/15+32_DQc-3y1k+01+0046000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_datase

height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+2pIlordhtto+00+0020000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+3bO_hDgvZBc+00+0013000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+-025GNtJ0oI+02+0086000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+-nnAdGpBxqk+00+0010000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_train/15+-GJ1zW0YCoM+00+0009000+1.jpg
height=360, width = 540
ImageID = Y2BB_extracted_dataset/train_for_train/15+2orxb-wN9Lk+00+0006000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+-90IS31mTso+00+0025000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+1lchW68PF2I+01+0016000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_train/15+28Dnk8Xy8qs+00+0004000+1.jpg
height=270, width = 480
ImageID = Y2BB_extracted_datase

height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+-HWsVPB5zUM+00+0029000+0.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+0e9fUBlGgu4+00+0120000+0.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+-93ox7o5khs+01+0103000+0.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+-wk7J5aKLB4+00+0120000+0.jpg
height=272, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+3gcvT89QZy4+00+0059000+0.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+1IcLjcXfZfI+06+0317000+0.jpg
height=360, width = 640
ImageID = Y2BB_extracted_dataset/train_for_train/15+2d0iQVxhQ_g+00+0081000+0.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+-ZY4b_yyA1c+00+0153000+0.jpg
height=320, width = 480
ImageID = Y2BB_extracted_dataset/train_for_train/15+1XKoeALBxt0+00+0039000+0.jpg
height=240, width = 320
ImageID = Y2BB_extracted_datase

In [16]:
train_vehicle = pd.read_csv(outputFileName)
train_vehicle.head()

,Y2BB_extracted_dataset/train_for_train/15+-6-4soyVvC4+01+0043000+1.jpg,260.64000000000004,143.4000024,319.20000000000005,190.199988,train
0,Y2BB_extracted_dataset/train_for_train/15+10X4...,38.4,79.680001,350.592,153.120010,train
1,Y2BB_extracted_dataset/train_for_train/15+1gXC...,0.0,85.199998,480.000,261.600012,train
2,Y2BB_extracted_dataset/train_for_train/15+3I5-...,0.0,0.000000,1280.000,440.400024,train
3,Y2BB_extracted_dataset/train_for_train/15+0qh9...,0.0,0.000000,192.000,85.199996,train
4,Y2BB_extracted_dataset/train_for_train/15+2iqa...,0.0,2.800000,320.000,153.999996,train


In [17]:
csvSourceFile = 'Y2BB_extracted_dataset/train_for_test/YTBB_train_testdata_metadata_v1.csv'
evalType = 'testing'
className = 'Train'
imageFolder = 'Y2BB_extracted_dataset/train_for_test/'
outputFileName = pre_process_y2bb_data(csvSourceFile,evalType, className,imageFolder)
print(f'outputFileName = {outputFileName}')

class_present = 300
class_absent = 0
ImageID = Y2BB_extracted_dataset/train_for_test/15+2lYnwkOVMlg+00+0071000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test/15+3MKBLFFsqHw+00+0016000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test/15+-025GNtJ0oI+02+0085000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test/15+3P_b80TKV_0+03+0106000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test/15+0jIZZsY3Q8A+00+0006000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_test/15+0POyH7D2a_U+01+0053000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/train_for_test/15+1H23VZeKRu8+00+0003000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test/15+16XQgOKblqs+01+0143000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_test/15+1e8KB4IqkpA+04+0306000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_d

height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_test/15+-p8PkIbU4lI+00+1896000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test/15+3Sx2QvjLSbk+01+0184000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test/15+2GLFCojapZ8+00+0017000+1.jpg
height=352, width = 640
ImageID = Y2BB_extracted_dataset/train_for_test/15+2i2R4fS-jQ4+00+0076000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/train_for_test/15+2HVWQ0SveT8+00+0016000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test/15+28DJ1IafNdA+03+0364000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test/15+3TnNb5Oc--8+13+0497000+1.jpg
height=360, width = 450
ImageID = Y2BB_extracted_dataset/train_for_test/15+1yhbvw1c9tk+00+0028000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test/15+2FMue4ul6oc+01+0125000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_

ImageID = Y2BB_extracted_dataset/train_for_test/15+3jiXJILAxWg+00+0031000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test/15+-bOySjRdVIw+01+0276000+1.jpg
height=262, width = 352
ImageID = Y2BB_extracted_dataset/train_for_test/15+1wXL_g0Nl8Q+01+0047000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test/15+0_2MLaEgk9Y+00+0133000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/train_for_test/15+07DiN3UALd0+00+0007000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test/15+-yMI3k70zIU+01+0060000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test/15+0n-lMeS_2fw+00+0001000+1.jpg
height=288, width = 352
ImageID = Y2BB_extracted_dataset/train_for_test/15+2Oau4Ai7QJc+01+0040000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test/15+2StaMMCS-_o+00+0048000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test/15+3IOcjLsBnXs+

ImageID = Y2BB_extracted_dataset/train_for_test/15+-EhbYnJ3-ao+00+0014000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test/15+-SFQS3TpR3s+00+0044000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test/15+3P_b80TKV_0+00+0008000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test/15+070CuYCc2Xw+02+0074000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_test/15+3cXVQvGU8O8+00+0045000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test/15+0HtmeOUMaGA+02+0089000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_test/15+-rOnmbSTcI8+00+0001000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_test/15+1kN4Y779u_o+00+0023000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_test/15+-yMI3k70zIU+01+0062000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test/15+2StaMMCS-_o+0

In [18]:
train_vehicle_test = pd.read_csv(outputFileName)
train_vehicle_test.head()

,Y2BB_extracted_dataset/train_for_test/15+2lYnwkOVMlg+00+0071000+1.jpg,0.0,0.0.1,444.0,315.0,train
0,Y2BB_extracted_dataset/train_for_test/15+3MKBL...,0.00,0.000000,480.00,294.599988,train
1,Y2BB_extracted_dataset/train_for_test/15+-025G...,0.00,0.000000,460.80,360.000000,train
2,Y2BB_extracted_dataset/train_for_test/15+3P_b8...,191.04,78.599999,480.00,252.000000,train
3,Y2BB_extracted_dataset/train_for_test/15+0jIZZ...,0.00,0.000000,320.00,134.000002,train
4,Y2BB_extracted_dataset/train_for_test/15+0POyH...,0.00,248.400000,1227.52,637.200000,train


In [19]:
csvSourceFile = 'Y2BB_extracted_dataset/train_for_validation/YTBB_train_validationdata_metadata_v1.csv'
evalType = 'validation'
className = 'Train'
imageFolder = 'Y2BB_extracted_dataset/train_for_validation/'
outputFileName = pre_process_y2bb_data(csvSourceFile,evalType, className,imageFolder)
print(f'outputFileName = {outputFileName}')

class_present = 200
class_absent = 0
ImageID = Y2BB_extracted_dataset/train_for_validation/15+-hlx_FW6Sk0+00+0019000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_validation/15+1gyflg5JA7E+00+0003000+1.jpg
height=360, width = 636
ImageID = Y2BB_extracted_dataset/train_for_validation/15+34H9BCu4aBs+00+0008000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_validation/15+0SkIi3YGKPI+00+0033000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_validation/15+3j3YyvV_OZU+00+0014000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_validation/15+2iqay12cCJc+04+0087000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_validation/15+0oGiHVOz6iY+02+0101000+1.jpg
height=360, width = 640
ImageID = Y2BB_extracted_dataset/train_for_validation/15+0DSELhvSLq4+00+0008000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_validation/15+-LkSgDlGI2w+00+0318000+1.j

height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_validation/15+3MKBLFFsqHw+00+0004000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_validation/15+-HE6kNgiWaI+00+0023000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_validation/15+-e6Rvdf43cQ+00+0011000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_validation/15+2Jzz9iLKooE+00+0015000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_validation/15+22NHRnlFer4+00+0159000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_validation/15+2BtG8h1du8g+00+0107000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_validation/15+1_VPW-l4DIg+00+0022000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_validation/15+2icEF4LKH1o+00+0004000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_validation/15+2GI4jRIGCuA+01+0149000+1.jpg
height=360

height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_validation/15+32_DQc-3y1k+01+0039000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_validation/15+1lchW68PF2I+00+0114000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_validation/15+3B6SlDNgCtU+00+0008000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_validation/15+01Qc5zMoY5A+00+0017000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_validation/15+1gyflg5JA7E+00+0011000+1.jpg
height=360, width = 636
ImageID = Y2BB_extracted_dataset/train_for_validation/15+0n-lMeS_2fw+00+0014000+1.jpg
height=288, width = 352
ImageID = Y2BB_extracted_dataset/train_for_validation/15+3M9zJQVwlRY+01+0058000+1.jpg
height=360, width = 636
ImageID = Y2BB_extracted_dataset/train_for_validation/15+3kGGyKkX91Q+00+0001000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_validation/15+-rOnmbSTcI8+00+0002000+1.jpg
height=240

In [20]:
train_vehicle_test = pd.read_csv(outputFileName)
train_vehicle_test.head()

,Y2BB_extracted_dataset/train_for_validation/15+-hlx_FW6Sk0+00+0019000+1.jpg,138.72,76.8000024,350.4,272.99998800000003,train
0,Y2BB_extracted_dataset/train_for_validation/15...,0.00,26.399999,512.616,360.000000,train
1,Y2BB_extracted_dataset/train_for_validation/15...,2.88,0.000000,480.000,315.000000,train
2,Y2BB_extracted_dataset/train_for_validation/15...,262.56,136.199999,300.480,180.000000,train
3,Y2BB_extracted_dataset/train_for_validation/15...,0.00,115.200000,397.440,271.199999,train
4,Y2BB_extracted_dataset/train_for_validation/15...,0.00,6.000000,320.000,152.400000,train


#### Preprocess Train and Test metadata CSV Files for Y2BB 'Airplane' class

In [21]:
csvSourceFile = 'Y2BB_extracted_dataset/airplane_for_train/YTBB_airplane_traindata_metadata_v1.csv'
evalType = 'training'
className = 'AirPlane'
imageFolder = 'Y2BB_extracted_dataset/airplane_for_train/'
outputFileName = pre_process_y2bb_data(csvSourceFile,evalType, className,imageFolder)
print(f'outputFileName = {outputFileName}')

class_present = 950
class_absent = 50
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+1RvBx5Yo1Q4+01+0063000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+2qK8HB73HhY+04+0079000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+243BzpfsPpo+00+0028000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+-6UiTYUvEiA+00+0029000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+3ITHoJbfmaI+01+0261000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+2uP2LM9Xu7U+00+0014000+1.jpg
height=360, width = 640
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+3XmgCmmT3q4+02+0081000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+-59Rg6_4eVo+01+0108000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+2GhhyUtXbjA+00+0210000+1.jpg
height=360, 

height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+3HKYAxj4igw+02+0241000+1.jpg
height=360, width = 450
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+29GvhbXmKqA+00+0135000+1.jpg
height=180, width = 240
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+02_Th396OSo+00+0133000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+3d6ce8vMzKM+00+0024000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+2LzlaZoD1xE+00+0134000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+-7J6f-dpvoE+00+0314000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+3NH_zd0vY50+00+0059000+1.jpg
height=240, width = 322
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+0bcMjuRPvb0+03+0254000+1.jpg
height=360, width = 524
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+0kK-SKGH7hc+00+0264000+1.jpg
height=360, width = 480
Im

ImageID = Y2BB_extracted_dataset/airplane_for_train/13+1xLDkTOw4SU+00+0012000+1.jpg
height=360, width = 640
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+-s6VK8Ry7TU+01+0393000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+0ieIPos7Ibg+00+0026000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+1qSB43qYs2A+00+0047000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+3C-L6xGSJAc+05+0299000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+2eZNnK-tcoA+01+0034000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+2X2HVi3Lgfw+00+0276000+1.jpg
height=360, width = 640
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+-UmHNz8mlbk+01+0188000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+12JU2-B4Xr8+00+0017000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_

height=360, width = 524
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+0h_pwJXQsc0+00+0187000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+1BdIJqoJ5Kw+01+0055000+1.jpg
height=216, width = 384
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+3VJpfL0Co3k+00+0009000+1.jpg
height=360, width = 640
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+2_TneWGgBV4+00+0137000+1.jpg
height=360, width = 640
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+0PTC_VWdxpE+00+0004000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+2dWblBkkccA+00+0024000+1.jpg
height=360, width = 640
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+2rlYRZwFyAk+02+0220000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+29GvhbXmKqA+01+0219000+1.jpg
height=180, width = 240
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+3HKYAxj4igw+05+0336000+1.jpg
height=360, width = 450
Ima

height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+-GPXMqq2028+00+0005000+1.jpg
height=360, width = 450
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+3C-L6xGSJAc+01+0135000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+16kdd9uPo9g+01+0251000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+11W5Bw4-pU4+08+0068000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+-uIFiysZVgw+00+0015000+1.jpg
height=360, width = 540
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+3rdS6OiSgDQ+00+0018000+1.jpg
height=270, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+2YieFvBSdfI+00+0016000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+010oXYICAwU+01+0061000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+-UmHNz8mlbk+04+0311000+1.jpg
height=360, width = 480
Im

...

height=360, width = 450
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+1oqDzceXOOM+02+0093000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+2vRmZafjTEI+00+0230000+1.jpg
height=360, width = 640
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+2qK8HB73HhY+02+0486000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+1L57LPMdXcs+02+0208000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+2lFrVWGxDws+00+0012000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+11W5Bw4-pU4+01+0196000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+0gf07SZa-jk+00+0164000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+3ITHoJbfmaI+01+0265000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+2hPqjpdQqv0+00+0006000+1.jpg
height=360, width = 480
I

ImageID = Y2BB_extracted_dataset/airplane_for_train/13+119psVhSObc+00+0032000+1.jpg
height=320, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+2hN4gTiYg68+02+0192000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+2Pi35Np_Rh0+00+0178000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+2Q5KjOBruiY+01+0157000+1.jpg
height=320, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+-txdJ796fYg+00+0017000+1.jpg
height=360, width = 640
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+0O-YR5X6xm4+00+0064000+1.jpg
height=144, width = 192
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+2X2HVi3Lgfw+01+0329000+1.jpg
height=360, width = 640
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+2T0G96mZH-U+00+0088000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+2ifMPiPpxKo+01+0505000+1.jpg
height=288, width = 352
ImageID = Y2BB_extracted_dat

height=240, width = 320
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+1vr9TKQx3JQ+01+0196000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+3d6ce8vMzKM+00+0021000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+2X2HVi3Lgfw+01+0325000+1.jpg
height=360, width = 640
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+3nX9lZ3aDBw+00+0053000+1.jpg
height=274, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+3ITHoJbfmaI+01+0262000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+-UmHNz8mlbk+03+0041000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+3fGcFpnPmMA+00+0085000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+2qK8HB73HhY+03+0525000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+2X2HVi3Lgfw+01+0321000+1.jpg
height=360, width = 640
I

height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+0iVbvXOQ5CM+01+0048000+0.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+3O5msUkCqzc+01+0320000+0.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+3NH_zd0vY50+00+0044000+0.jpg
height=240, width = 322
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+-rDsinRUuw0+00+0086000+0.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+03T1sYnfqwk+00+0118000+0.jpg
height=360, width = 640
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+12oRIoO4F1w+08+0627000+0.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+3-9XRuye3ww+02+0413000+0.jpg
height=360, width = 472
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+-FOKM9iC1Io+00+0227000+0.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+-fmv5H-yyoY+00+0010000+0.jpg
height=720, width = 1280
Im

height=240, width = 320
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+3cttxurdemI+02+0063000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+0Oi63sISrFI+00+0010000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+1L57LPMdXcs+04+0277000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+1lWXMjTdFxI+01+0040000+1.jpg
height=360, width = 482
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+36ymO829OaU+02+0512000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+-lxhXRXXvzc+06+0094000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+-ZRXNJQXEM4+00+0350000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+0UsnSCE9nD8+00+0116000+1.jpg
height=360, width = 640
ImageID = Y2BB_extracted_dataset/airplane_for_train/13+16m4uqs4Xdc+00+0039000+1.jpg
height=360, width = 480
Ima

In [22]:
airplane_vehicle = pd.read_csv(outputFileName)
airplane_vehicle.head()

,Y2BB_extracted_dataset/airplane_for_train/13+1RvBx5Yo1Q4+01+0063000+1.jpg,132.95999999999998,160.79999759999998,303.84000000000003,211.19999040000002,airplane
0,Y2BB_extracted_dataset/airplane_for_train/13+2...,0.00,159.599995,1280.00,649.199981,airplane
1,Y2BB_extracted_dataset/airplane_for_train/13+2...,253.44,141.000005,398.40,187.200000,airplane
2,Y2BB_extracted_dataset/airplane_for_train/13+-...,0.00,16.200000,480.00,228.600000,airplane
3,Y2BB_extracted_dataset/airplane_for_train/13+3...,334.08,97.800005,412.32,138.600000,airplane
4,Y2BB_extracted_dataset/airplane_for_train/13+2...,244.48,90.600001,478.08,195.600006,airplane


In [23]:
csvSourceFile = 'Y2BB_extracted_dataset/airplane_for_test/YTBB_airplane_testdata_metadata_v1.csv'
evalType = 'testing'
className = 'AirPlane'
imageFolder = 'Y2BB_extracted_dataset/airplane_for_test/'
outputFileName = pre_process_y2bb_data(csvSourceFile,evalType, className,imageFolder)
print(f'outputFileName = {outputFileName}')

class_present = 300
class_absent = 0
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+0KVW3yvNVH4+00+0132000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+-pQFjID-vHo+00+0406000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+-lxhXRXXvzc+03+1413000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+3HKYAxj4igw+05+0343000+1.jpg
height=360, width = 450
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+1FusH4o2PN4+00+0022000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+018trrxTJhQ+00+0011000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+12oRIoO4F1w+07+0597000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+243BzpfsPpo+00+0020000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+3O5msUkCqzc+00+0034000+1.jpg
height=240, width = 320

height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+-10DMgsYgmg+00+0002000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+0i3tuUNNgVc+00+0021000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+1dUFNzIMBDo+00+0047000+1.jpg
height=360, width = 640
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+3Bk1DMmrwKM+01+0016000+1.jpg
height=360, width = 540
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+1Z_0ACrSAHg+03+0302000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+0Ynq5IFj6PE+00+0370000+1.jpg
height=320, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+3G591E3xlT8+04+0090000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+2hPqjpdQqv0+00+0012000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+1ml7l5atuIE+02+0402000+1.jpg
height=360, width = 640
ImageID = 

ImageID = Y2BB_extracted_dataset/airplane_for_test/13+2EchfuyQzbA+00+0011000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+3C-L6xGSJAc+00+0006000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+-tyNzOHJ3gw+00+0028000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+030iIeoa-tY+01+0049000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+1QVNYiqyx2E+00+0010000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+1C7xA1TdsIE+00+0037000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+2u7Fjf6OFRQ+00+0172000+1.jpg
height=320, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+-e5nPWhRwdg+00+0166000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+0V2ixO7vrjs+01+0144000+1.jpg
height=360, width = 492
ImageID = Y2BB_extracted_dataset/ai

height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+3Bk1DMmrwKM+00+0127000+1.jpg
height=360, width = 540
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+0V2ixO7vrjs+00+0005000+1.jpg
height=360, width = 492
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+-lxhXRXXvzc+06+0104000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_test/13+-VFtlwYL9FQ+00+0043000+1.jpg
height=360, width = 480
outputFileName = y2bb_testing_AirPlane_annotate.txt


In [24]:
airplane_vehicle_test = pd.read_csv(outputFileName)
airplane_vehicle_test.head()

,Y2BB_extracted_dataset/airplane_for_test/13+0KVW3yvNVH4+00+0132000+1.jpg,334.08000000000004,98.40000239999999,1280.0,296.39999520000003,airplane
0,Y2BB_extracted_dataset/airplane_for_test/13+-p...,0.00,149.400000,480.00,337.200001,airplane
1,Y2BB_extracted_dataset/airplane_for_test/13+-l...,117.12,153.600005,432.48,247.200001,airplane
2,Y2BB_extracted_dataset/airplane_for_test/13+3H...,179.55,201.000002,321.30,259.800012,airplane
3,Y2BB_extracted_dataset/airplane_for_test/13+1F...,96.64,91.200000,256.96,161.600004,airplane
4,Y2BB_extracted_dataset/airplane_for_test/13+01...,0.00,0.000000,480.00,286.200000,airplane


In [25]:
csvSourceFile = 'Y2BB_extracted_dataset/airplane_for_validation/YTBB_airplane_validationdata_metadata_v1.csv'
evalType = 'validation'
className = 'AirPlane'
imageFolder = 'Y2BB_extracted_dataset/airplane_for_validation/'
outputFileName = pre_process_y2bb_data(csvSourceFile,evalType, className,imageFolder)
print(f'outputFileName = {outputFileName}')

class_present = 200
class_absent = 0
ImageID = Y2BB_extracted_dataset/airplane_for_validation/13+3V_tgGdzXbc+00+0168000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_validation/13+2qK8HB73HhY+00+0006000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_validation/13+-ZRXNJQXEM4+00+0346000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_validation/13+1op49Y24Xdk+00+0040000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_validation/13+2xrFXftTvuA+00+0042000+1.jpg
height=304, width = 400
ImageID = Y2BB_extracted_dataset/airplane_for_validation/13+1qSB43qYs2A+00+0045000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_validation/13+030iIeoa-tY+02+0085000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_validation/13+3HKYAxj4igw+02+0232000+1.jpg
height=360, width = 450
ImageID = Y2BB_extracted_dataset/airplane_for_validati

height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_validation/13+28AfvuPIsSw+00+0008000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_validation/13+3sDWzq7nyYk+00+0013000+1.jpg
height=360, width = 636
ImageID = Y2BB_extracted_dataset/airplane_for_validation/13+11W5Bw4-pU4+02+0239000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_validation/13+0JpmdKkOQTM+00+0237000+1.jpg
height=360, width = 640
ImageID = Y2BB_extracted_dataset/airplane_for_validation/13+3RYPXRkCspc+01+0080000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_validation/13+2T0G96mZH-U+00+0089000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_validation/13+0VDgNRAgdIY+01+0076000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_validation/13+03T1sYnfqwk+00+0111000+1.jpg
height=360, width = 640
ImageID = Y2BB_extracted_dataset/airplane_for_validation/13+3QQfLaOS

height=360, width = 640
ImageID = Y2BB_extracted_dataset/airplane_for_validation/13+1mncUKMFWmk+03+0301000+1.jpg
height=288, width = 384
ImageID = Y2BB_extracted_dataset/airplane_for_validation/13+3Hy7pB1SAnM+00+0025000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_validation/13+2yH3VzXWIZ8+00+0134000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_validation/13+0d5YLn14gKQ+00+0018000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_validation/13+2WvWsoa4Tq8+02+0084000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_validation/13+2ifMPiPpxKo+01+0496000+1.jpg
height=288, width = 352
ImageID = Y2BB_extracted_dataset/airplane_for_validation/13+2k8iuasNwTA+00+0074000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_validation/13+2O7Ahxqr9rI+01+0177000+1.jpg
height=288, width = 384
ImageID = Y2BB_extracted_dataset/airplane_for_validation/13+-tyNzOHJ3gw

In [26]:
airplane_vehicle_test = pd.read_csv(outputFileName)
airplane_vehicle_test.head()

,Y2BB_extracted_dataset/airplane_for_validation/13+3V_tgGdzXbc+00+0168000+1.jpg,409.6,219.6,1191.6799999999998,409.19997600000005,airplane
0,Y2BB_extracted_dataset/airplane_for_validation...,144.64,264.000002,1103.36,596.399976,airplane
1,Y2BB_extracted_dataset/airplane_for_validation...,136.80,145.200002,217.92,169.200000,airplane
2,Y2BB_extracted_dataset/airplane_for_validation...,0.00,0.000000,917.76,720.000000,airplane
3,Y2BB_extracted_dataset/airplane_for_validation...,0.00,0.000000,400.00,304.000000,airplane
4,Y2BB_extracted_dataset/airplane_for_validation...,88.32,297.599998,1108.48,494.400002,airplane


#### Preprocess Train and Test metadata CSV Files for COCO 'Train' class - large size images

In [8]:
csvSourceFile = 'COCO_extracted_dataset/large_train_for_test/train_for_test_large.csv'
evalType = 'testing_LargeImage'
className = 'Train'
imageFolder = 'COCO_extracted_dataset/large_train_for_test/'
outputFileName = pre_process_coco_data(csvSourceFile,evalType, className,imageFolder)
print(f'outputFileName = {outputFileName}')

class_present = 289
class_absent = 0
outputFileName = coco_testing_LargeImage_Train_annotate.txt


In [9]:
train_vehicle = pd.read_csv(outputFileName)
train_vehicle.head()

,COCO_extracted_dataset/large_train_for_test/COCO_val2014_000000223157.jpg,201.17,122.88,315.45,206.91,train
0,COCO_extracted_dataset/large_train_for_test/CO...,0.00,291.64,89.92,401.55,train
1,COCO_extracted_dataset/large_train_for_test/CO...,277.00,188.87,366.11,301.22,train
2,COCO_extracted_dataset/large_train_for_test/CO...,257.75,143.77,515.26,182.74,train
3,COCO_extracted_dataset/large_train_for_test/CO...,136.10,236.77,257.55,323.13,train
4,COCO_extracted_dataset/large_train_for_test/CO...,65.93,298.81,339.90,337.33,train


#### Preprocess Train and Test metadata CSV Files for COCO 'Train' class - medium size images

In [10]:
csvSourceFile = 'COCO_extracted_dataset/medium_train_for_test/train_for_test_medium.csv'
evalType = 'testing_MediumImage'
className = 'Train'
imageFolder = 'COCO_extracted_dataset/medium_train_for_test/'
outputFileName = pre_process_coco_data(csvSourceFile,evalType, className,imageFolder)
print(f'outputFileName = {outputFileName}')

class_present = 11
class_absent = 0
outputFileName = coco_testing_MediumImage_Train_annotate.txt


In [11]:
train_vehicle = pd.read_csv(outputFileName)
train_vehicle.head()

,COCO_extracted_dataset/medium_train_for_test/COCO_val2014_000000535591.jpg,309.84,280.15,342.76,315.01,train
0,COCO_extracted_dataset/medium_train_for_test/C...,350.87,376.11,435.45,399.93,train
1,COCO_extracted_dataset/medium_train_for_test/C...,414.47,236.10,462.74,279.37,train
2,COCO_extracted_dataset/medium_train_for_test/C...,350.78,186.50,394.52,234.98,train
3,COCO_extracted_dataset/medium_train_for_test/C...,365.56,161.25,389.78,253.31,train
4,COCO_extracted_dataset/medium_train_for_test/C...,452.73,350.62,500.43,429.40,train


#### Preprocess Train and Test metadata CSV Files for COCO 'Train' class - small size images

In [12]:
csvSourceFile = 'COCO_extracted_dataset/small_train_for_test/train_for_test_small.csv'
evalType = 'testing_SmallImage'
className = 'Train'
imageFolder = 'COCO_extracted_dataset/small_train_for_test/'
outputFileName = pre_process_coco_data(csvSourceFile,evalType, className,imageFolder)
print(f'outputFileName = {outputFileName}')

class_present = 3
class_absent = 0
outputFileName = coco_testing_SmallImage_Train_annotate.txt


In [13]:
train_vehicle = pd.read_csv(outputFileName)
train_vehicle.head()

,COCO_extracted_dataset/small_train_for_test/COCO_val2014_000000245051.jpg,250.51,303.55,279.87,316.21,train
0,COCO_extracted_dataset/small_train_for_test/CO...,79.22,405.86,114.24,424.37,train
1,COCO_extracted_dataset/small_train_for_test/CO...,364.76,301.35,387.69,331.59,train


#### Preprocess Train and Test metadata CSV Files for COCO 'Airplane' class - small size images

In [14]:
csvSourceFile = 'COCO_extracted_dataset/large_airplane_for_test/airplane_for_test_large.csv'
evalType = 'testing_LargeImage'
className = 'Airplane'
imageFolder = 'COCO_extracted_dataset/large_airplane_for_test/'
outputFileName = pre_process_coco_data(csvSourceFile,evalType, className,imageFolder)
print(f'outputFileName = {outputFileName}')

class_present = 236
class_absent = 0
outputFileName = coco_testing_LargeImage_Airplane_annotate.txt


In [15]:
Airplane_vehicle = pd.read_csv(outputFileName)
Airplane_vehicle.head()

,COCO_extracted_dataset/large_airplane_for_test/COCO_val2014_000000283495.jpg,294.24,65.05,432.86,135.42,airplane
0,COCO_extracted_dataset/large_airplane_for_test...,132.21,48.77,288.29,112.76,airplane
1,COCO_extracted_dataset/large_airplane_for_test...,16.05,49.32,166.65,122.72,airplane
2,COCO_extracted_dataset/large_airplane_for_test...,227.65,214.41,399.46,279.91,airplane
3,COCO_extracted_dataset/large_airplane_for_test...,522.07,45.57,638.56,149.12,airplane
4,COCO_extracted_dataset/large_airplane_for_test...,339.22,201.86,461.67,301.14,airplane


#### Preprocess Train and Test metadata CSV Files for COCO 'Airplane' class - medium size images

In [16]:
csvSourceFile = 'COCO_extracted_dataset/medium_airplane_for_test/airplane_for_test_medium.csv'
evalType = 'testing_MediumImage'
className = 'Airplane'
imageFolder = 'COCO_extracted_dataset/medium_airplane_for_test/'
outputFileName = pre_process_coco_data(csvSourceFile,evalType, className,imageFolder)
print(f'outputFileName = {outputFileName}')

class_present = 48
class_absent = 0
outputFileName = coco_testing_MediumImage_Airplane_annotate.txt


In [17]:
Airplane_vehicle = pd.read_csv(outputFileName)
Airplane_vehicle.head()

,COCO_extracted_dataset/medium_airplane_for_test/COCO_val2014_000000564061.jpg,318.99,24.22,384.22,40.67,airplane
0,COCO_extracted_dataset/medium_airplane_for_tes...,162.51,243.82,198.93,275.37,airplane
1,COCO_extracted_dataset/medium_airplane_for_tes...,329.36,325.26,387.79,345.39,airplane
2,COCO_extracted_dataset/medium_airplane_for_tes...,182.12,281.40,221.59,314.71,airplane
3,COCO_extracted_dataset/medium_airplane_for_tes...,8.73,97.47,68.15,120.36,airplane
4,COCO_extracted_dataset/medium_airplane_for_tes...,213.18,142.70,257.86,175.76,airplane


#### Preprocess Train and Test metadata CSV Files for COCO 'Airplane' class - small size images

In [18]:
csvSourceFile = 'COCO_extracted_dataset/small_airplane_for_test/airplane_for_test_small.csv'
evalType = 'testing_SmallImage'
className = 'Airplane'
imageFolder = 'COCO_extracted_dataset/small_airplane_for_test/'
outputFileName = pre_process_coco_data(csvSourceFile,evalType, className,imageFolder)
print(f'outputFileName = {outputFileName}')

class_present = 16
class_absent = 0
outputFileName = coco_testing_SmallImage_Airplane_annotate.txt


In [19]:
Airplane_vehicle = pd.read_csv(outputFileName)
Airplane_vehicle.head()

,COCO_extracted_dataset/small_airplane_for_test/COCO_val2014_000000138755.jpg,352.77,228.71,366.94,232.36,airplane
0,COCO_extracted_dataset/small_airplane_for_test...,188.40,16.65,202.39,21.08,airplane
1,COCO_extracted_dataset/small_airplane_for_test...,422.67,108.70,444.58,118.57,airplane
2,COCO_extracted_dataset/small_airplane_for_test...,471.77,226.32,503.99,236.31,airplane
3,COCO_extracted_dataset/small_airplane_for_test...,327.17,229.91,363.33,239.26,airplane
4,COCO_extracted_dataset/small_airplane_for_test...,379.75,124.30,407.25,139.13,airplane


#### Preprocess Train and Test metadata CSV Files for YTBB 'Train' class - large size images

In [20]:
csvSourceFile = 'Y2BB_extracted_dataset/train_for_test_large/YTBB_train_testdata_metadata_v1_large.csv' 
evalType = 'testing_LargeImage'
className = 'Train'
imageFolder = 'Y2BB_extracted_dataset/train_for_test_large/'
outputFileName = pre_process_y2bb_data(csvSourceFile,evalType, className,imageFolder)
print(f'outputFileName = {outputFileName}')

class_present = 277
class_absent = 0
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+-025GNtJ0oI+00+0011000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+-025GNtJ0oI+02+0085000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+-6dnETcotmM+00+0006000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+-771AUMRGi0+00+0003000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+-9Kf0njIktU+00+0015000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+-BwXEDixxNM+00+0013000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+-EhbYnJ3-ao+00+0014000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+-EuXs7Mp8hM+00+0031000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+-EuXs7Mp8hM+00+0037000+1.j

height=720, width = 1280
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+0_2MLaEgk9Y+00+0133000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+0e62YRaGzro+00+0007000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+0eWtYZVw73g+00+0019000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+0j6RR7_Y8Ow+01+0074000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+0jIZZsY3Q8A+00+0006000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+0ldxVX7sSvE+00+0006000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+0mJwl9SmdVU+00+0014000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+0n-lMeS_2fw+00+0001000+1.jpg
height=288, width = 352
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+0n-lMeS_2fw+02+0142000+1.jpg
height=

height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+2KL8RWSy1_g+02+0395000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+2LenaoY9oYs+00+0014000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+2LenaoY9oYs+00+0021000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+2Mhk6ktdBgs+00+0030000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+2Oau4Ai7QJc+01+0040000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+2QFHMVsluNs+00+0006000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+2QFHMVsluNs+00+0016000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+2RPLt0vxnvc+00+0005000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+2RdNUMTtm60+00+0021000+1.jpg
height=240

ImageID = Y2BB_extracted_dataset/train_for_test_large/15+3TnNb5Oc--8+12+0463000+1.jpg
height=360, width = 450
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+3TnNb5Oc--8+12+0465000+1.jpg
height=360, width = 450
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+3TnNb5Oc--8+13+0497000+1.jpg
height=360, width = 450
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+3UW4Ze5SAog+00+0005000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+3X3-ZoE3yHI+00+0015000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+3X3-ZoE3yHI+01+0044000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+3X3-ZoE3yHI+02+0097000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+3X68YQk3WpE+00+0006000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test_large/15+3XM9XQ5aH7g+00+0016000+1.jpg
height=240, width = 320
ImageID = 

In [21]:
train_vehicle = pd.read_csv(outputFileName)
train_vehicle.head()

,Y2BB_extracted_dataset/train_for_test_large/15+-025GNtJ0oI+00+0011000+1.jpg,0.0,83.9999988,480.0,212.39999999999998,train
0,Y2BB_extracted_dataset/train_for_test_large/15...,0.0,0.000000,460.80,360.000000,train
1,Y2BB_extracted_dataset/train_for_test_large/15...,412.8,40.800002,480.00,360.000000,train
2,Y2BB_extracted_dataset/train_for_test_large/15...,0.0,55.999999,320.00,240.000000,train
3,Y2BB_extracted_dataset/train_for_test_large/15...,0.0,33.600000,368.16,289.200002,train
4,Y2BB_extracted_dataset/train_for_test_large/15...,156.0,100.800000,360.48,255.600000,train


#### Preprocess Train and Test metadata CSV Files for YTBB 'Train' class - medium size images

In [22]:
csvSourceFile = 'Y2BB_extracted_dataset/train_for_test_medium/YTBB_train_testdata_metadata_v1_medium.csv' 
evalType = 'testing_MediumImage'
className = 'Train'
imageFolder = 'Y2BB_extracted_dataset/train_for_test_medium/'
outputFileName = pre_process_y2bb_data(csvSourceFile,evalType, className,imageFolder)
print(f'outputFileName = {outputFileName}')

class_present = 17
class_absent = 0
ImageID = Y2BB_extracted_dataset/train_for_test_medium/15+-6-4soyVvC4+01+0039000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test_medium/15+-hj2q9utBhs+02+0036000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_test_medium/15+-yMI3k70zIU+00+0019000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test_medium/15+0DwhUrmEAws+00+0076000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test_medium/15+16XQgOKblqs+01+0143000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_test_medium/15+1_xfocbatMQ+01+0029000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test_medium/15+1e8KB4IqkpA+05+0335000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test_medium/15+1n_jnfJVPzs+00+0012000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_test_medium/15+1n_jnfJVPzs+00+001

In [23]:
train_vehicle = pd.read_csv(outputFileName)
train_vehicle.head()

,Y2BB_extracted_dataset/train_for_test_medium/15+-6-4soyVvC4+01+0039000+1.jpg,213.6,132.6000024,281.76000000000005,194.4,train
0,Y2BB_extracted_dataset/train_for_test_medium/1...,175.68,126.399994,310.72,153.600000,train
1,Y2BB_extracted_dataset/train_for_test_medium/1...,269.76,93.600000,341.76,142.800005,train
2,Y2BB_extracted_dataset/train_for_test_medium/1...,276.48,168.599999,317.76,198.000000,train
3,Y2BB_extracted_dataset/train_for_test_medium/1...,190.72,86.400000,315.20,146.000004,train
4,Y2BB_extracted_dataset/train_for_test_medium/1...,253.44,100.800000,319.68,146.400001,train


#### Preprocess Train and Test metadata CSV Files for YTBB 'Train' class - small size images

In [38]:
csvSourceFile = 'Y2BB_extracted_dataset/train_for_test_small/YTBB_train_testdata_metadata_v1_small.csv' 
evalType = 'testing_SmallImage'
className = 'Train'
imageFolder = 'Y2BB_extracted_dataset/train_for_test_small/'
outputFileName = pre_process_y2bb_data(csvSourceFile,evalType, className,imageFolder)
print(f'outputFileName = {outputFileName}')

class_present = 5
class_absent = 0
ImageID = Y2BB_extracted_dataset/train_for_test_small/15+-rOnmbSTcI8+00+0001000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_test_small/15+0CVUom4VkXA+00+0027000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test_small/15+0JKG5cXe_sc+00+0001000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/train_for_test_small/15+1MiZxq3SJxg+00+0269000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/train_for_test_small/15+3h18x3Qe6so+00+0359000+1.jpg
height=360, width = 480
outputFileName = y2bb_testing_SmallImage_Train_annotate.txt


In [39]:
train_vehicle = pd.read_csv(outputFileName)
train_vehicle.head()

,Y2BB_extracted_dataset/train_for_test_small/15+-rOnmbSTcI8+00+0001000+1.jpg,76.16000000000001,106.39999920000001,94.72000000000001,120.7999992,train
0,Y2BB_extracted_dataset/train_for_test_small/15...,135.36,204.599988,157.44,216.000000,train
1,Y2BB_extracted_dataset/train_for_test_small/15...,118.08,212.400000,139.68,235.199988,train
2,Y2BB_extracted_dataset/train_for_test_small/15...,145.60,96.400001,174.40,125.599992,train
3,Y2BB_extracted_dataset/train_for_test_small/15...,283.20,107.400006,304.80,142.200000,train


#### Preprocess Train and Test metadata CSV Files for YTBB 'Airplane' class - large size images

In [31]:
csvSourceFile = 'Y2BB_extracted_dataset/airplane_for_test_large/YTBB_airplane_testdata_metadata_v1_large.csv' 
evalType = 'testing_LargeImage'
className = 'AirPlane'
imageFolder = 'Y2BB_extracted_dataset/airplane_for_test_large/'
outputFileName = pre_process_y2bb_data(csvSourceFile,evalType, className,imageFolder)
print(f'outputFileName = {outputFileName}')

class_present = 278
class_absent = 0
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+-0oOCh2swmU+00+0030000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+-10DMgsYgmg+00+0002000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+-10DMgsYgmg+00+0007000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+-10DMgsYgmg+01+0029000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+-10DMgsYgmg+01+0032000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+-10DMgsYgmg+01+0041000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+-59Rg6_4eVo+01+0118000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+-59Rg6_4eVo+03+0081000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/airplane_for_test_lar

height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+0lVylP-SMAs+00+0021000+1.jpg
height=360, width = 640
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+0lVylP-SMAs+00+0024000+1.jpg
height=360, width = 640
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+0nNVzuYeLCE+00+0085000+1.jpg
height=360, width = 540
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+0q0PfpBsKVo+01+0029000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+0s2-9EsxiUw+00+0002000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+0s2-9EsxiUw+00+0010000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+0ynaDW3cho4+00+0015000+1.jpg
height=262, width = 352
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+0ynaDW3cho4+01+0104000+1.jpg
height=262, width = 352
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+10sbVWvtyk

height=240, width = 320
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+2LzlaZoD1xE+01+0051000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+2O7Ahxqr9rI+01+0165000+1.jpg
height=288, width = 384
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+2O7Ahxqr9rI+01+0173000+1.jpg
height=288, width = 384
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+2O7Ahxqr9rI+03+0237000+1.jpg
height=288, width = 384
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+2O7Ahxqr9rI+04+0276000+1.jpg
height=288, width = 384
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+2O7Ahxqr9rI+09+0059000+1.jpg
height=288, width = 384
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+2Q5KjOBruiY+01+0165000+1.jpg
height=320, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+2Q5KjOBruiY+02+0078000+1.jpg
height=320, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+2Q5KjOBruiY+

ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+3cd9lQQ3gpE+00+0044000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+3cttxurdemI+00+0130000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+3cttxurdemI+03+0092000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+3cttxurdemI+03+0094000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+3cttxurdemI+03+0096000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+3d46e7ZIgCs+00+0233000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+3d5sJ6FadFQ+00+0010000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+3d6ce8vMzKM+00+0029000+1.jpg
height=720, width = 1280
ImageID = Y2BB_extracted_dataset/airplane_for_test_large/13+3eJOcCwWSxQ+00+0040000+1.jpg
heigh

In [32]:
airplane_test = pd.read_csv(outputFileName)
airplane_test.head()

,Y2BB_extracted_dataset/airplane_for_test_large/13+-0oOCh2swmU+00+0030000+1.jpg,0.0,87.60000240000001,480.0,276.59998800000005,airplane
0,Y2BB_extracted_dataset/airplane_for_test_large...,304.64,72.000000,1276.16,470.399976,airplane
1,Y2BB_extracted_dataset/airplane_for_test_large...,185.60,51.599999,1224.96,464.400000,airplane
2,Y2BB_extracted_dataset/airplane_for_test_large...,0.00,120.000002,962.56,590.400000,airplane
3,Y2BB_extracted_dataset/airplane_for_test_large...,0.00,129.600000,1084.16,580.800024,airplane
4,Y2BB_extracted_dataset/airplane_for_test_large...,268.80,240.000005,1058.56,554.400000,airplane


#### Preprocess Train and Test metadata CSV Files for YTBB 'Airplane' class - medium size images

In [28]:
csvSourceFile = 'Y2BB_extracted_dataset/airplane_for_test_medium/YTBB_airplane_testdata_metadata_v1_medium.csv' 
evalType = 'testing_MediumImage'
className = 'AirPlane'
imageFolder = 'Y2BB_extracted_dataset/airplane_for_test_medium/'
outputFileName = pre_process_y2bb_data(csvSourceFile,evalType, className,imageFolder)
print(f'outputFileName = {outputFileName}')

class_present = 19
class_absent = 0
ImageID = Y2BB_extracted_dataset/airplane_for_test_medium/13+-9AbB-h7c1Y+01+0472000+1.jpg
height=240, width = 322
ImageID = Y2BB_extracted_dataset/airplane_for_test_medium/13+-e5nPWhRwdg+00+0166000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_test_medium/13+-fYp8Dr3wR0+03+0359000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/airplane_for_test_medium/13+0V2ixO7vrjs+01+0144000+1.jpg
height=360, width = 492
ImageID = Y2BB_extracted_dataset/airplane_for_test_medium/13+0kK-SKGH7hc+00+0259000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_test_medium/13+1FusH4o2PN4+00+0017000+1.jpg
height=240, width = 320
ImageID = Y2BB_extracted_dataset/airplane_for_test_medium/13+1ts_Fz855WE+01+0072000+1.jpg
height=358, width = 640
ImageID = Y2BB_extracted_dataset/airplane_for_test_medium/13+227HZwlTfIk+00+0030000+1.jpg
height=360, width = 480
ImageID = Y2BB_extracted_dataset/airplane_for_test_m

In [29]:
airplane_test = pd.read_csv(outputFileName)
airplane_test.head()

,Y2BB_extracted_dataset/airplane_for_test_medium/13+-9AbB-h7c1Y+01+0472000+1.jpg,111.412,63.6,219.28199999999998,115.59999599999999,airplane
0,Y2BB_extracted_dataset/airplane_for_test_mediu...,176.64,151.200000,357.60,189.000000,airplane
1,Y2BB_extracted_dataset/airplane_for_test_mediu...,149.12,118.800000,238.08,156.000000,airplane
2,Y2BB_extracted_dataset/airplane_for_test_mediu...,132.84,144.600001,258.30,184.799988,airplane
3,Y2BB_extracted_dataset/airplane_for_test_mediu...,127.68,128.399994,229.92,179.399999,airplane
4,Y2BB_extracted_dataset/airplane_for_test_mediu...,129.60,116.400000,250.24,166.399992,airplane


#### Preprocess Train and Test metadata CSV Files for YTBB 'Airplane' class - small size images

In [34]:
csvSourceFile = 'Y2BB_extracted_dataset/airplane_for_test_small/YTBB_airplane_testdata_metadata_v1_small.csv' 
evalType = 'testing_SmallImage'
className = 'AirPlane'
imageFolder = 'Y2BB_extracted_dataset/airplane_for_test_small/'
outputFileName = pre_process_y2bb_data(csvSourceFile,evalType, className,imageFolder)
print(f'outputFileName = {outputFileName}')

class_present = 2
class_absent = 0
ImageID = Y2BB_extracted_dataset/airplane_for_test_small/13+1ofKqFardDM+01+0030000+1.jpg
height=288, width = 384
ImageID = Y2BB_extracted_dataset/airplane_for_test_small/13+2ifMPiPpxKo+00+0464000+1.jpg
height=288, width = 352
outputFileName = y2bb_testing_SmallImage_AirPlane_annotate.txt


In [35]:
airplane_test = pd.read_csv(outputFileName)
airplane_test.head()

,Y2BB_extracted_dataset/airplane_for_test_small/13+1ofKqFardDM+01+0030000+1.jpg,177.79199999999997,101.75999616,190.848,112.80000384,airplane
0,Y2BB_extracted_dataset/airplane_for_test_small...,145.376,148.32,175.296,163.199998,airplane
